In [1]:
%pylab inline

from __future__ import print_function
from __future__ import division

import sklearn
import numpy
import scipy
import pandas
import matplotlib
import seaborn

import json
import re

from matplotlib import pyplot
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [2]:
rcParams['figure.figsize'] = 15, 8

Los mails se componen de varios campos entre los que está el emisor, el destinatario y el mensaje. Primero, procesaremos todos los mensajes para identificar cada uno de esos campos y su correspondiente valor.

Cosas que quiero testear si se cumplen para todos los mensajes
--------------------------------------------------------------

- todos los campos tienen un nombre excepto el mensaje
- separador de campos: '\r\n'
- separador del mensaje del resto de los campos: 2*sep_campos
- separador entre nombre de campo y campo: ':'

Preguntas
---------

- el mensaje siempre es el último campo ?
- Los mensajes están mezclados? Puedo usar el tail como test?


Atributos posibles
------------------

- qué están atributos presentes
- orden de los atributos
- largo del mensaje
- propiedades sobre ciertos atributos:
    - emisor
        - dominio de la dirección
        - largo de la dirección
    - destinatario
        - cantidad direcciones
    - asunto
        - largo
        - análisis parecido al mensaje: palabras presentes, etc.
- si hay manera de identificar si el mail fue forwardeado creo que sería bueno, casi nadie debe forwardea spam


In [4]:
linesep = '\r\n'
bodysep = linesep * 2 

def valid(msg):
    return bodysep in msg

def parse2(messages):
    return [
        {
            x.split(':')[0]: x.split(':')[1]
            for x in k.split(bodysep)[0].split(linesep) + [u'message:{}'.format(k.split(bodysep)[1])]
            if ':' in x
        }
        for k in [m.replace('\n\r\t', '') for m in messages]
    ]

def parse(messages):
    parsed = []
    for k in messages:
        for x in k.split(linesep):
            parsed.append((x.split(':')[0], x.split(':')[1]) if ':' in x else x)
                
    return parsed

In [3]:
ham_json = json.load(open('data/ham_train.json'))
ham = parse(ham_json)
ham[:20]

ValueError: No JSON object could be decoded

In [7]:
spam_json = json.load(open('data/spam_train.json'))
spam = parse(x for x in spam_json if valid(x))
spam[:20]

ValueError: No JSON object could be decoded

In [ ]:
def key_barplot(p, limit = 0):
    from operator import itemgetter
    keys = __builtin__.sum((Counter(x.keys()) for x in p), Counter())
    vals = sorted(((x, y) for x, y in keys.iteritems() if y > limit), key = itemgetter(1))
    pyplot.xticks(rotation = 90)
    seaborn.barplot(map(itemgetter(0), vals), map(itemgetter(1), vals))

In [ ]:
key_barplot(ham, limit = 5)

In [ ]:
key_barplot(spam, limit = 0)

# Testing

In [43]:
import re

In [50]:
def parse(t):
    return [
        # tuple(x.rsplit(':', 1)) if re.match(r'\S*:', x) else ('message', x)
        tuple(x.rsplit(':', 1))
        for x in t.split('\r\n')
    ]

In [51]:
parse(ham_json[0])

[(u'message-id', u' <13247446.1075856567056.javamail.evans#####>'),
 (u'date: mon, 26 jun 2000 14:22', u'00 -0700 (pdt)'),
 (u'from', u' vkaminski#####'),
 (u'to', u' chris#####'),
 (u'subject: re', u' eprm articles'),
 (u'cc', u' vkaminski#####, vkamins#####'),
 (u'mime-version', u' 1.0'),
 (u'content-type', u' text/plain; charset=us-ascii'),
 (u'content-transfer-encoding', u' 7bit'),
 (u'bcc', u' vkaminski#####, vkamins#####'),
 (u'x-from', u' vkaminski#####'),
 (u'x-to', u' chris#####'),
 (u'x-cc', u' vkaminski#####, vkamins#####'),
 (u'x-bcc', u' '),
 (u'x-origin', u' kaminski-v'),
 (u'x-filename', u' vkamins.nsf'),
 (u'',),
 (u"chris,\n\nthanks for the invitation. the evening of july the 18th is fine  with me.\n\nthe list looks fine and it can be easily expanded, if the first set of\narticles is well received. i shall prepare a list of topics that occupy us\nevery day and that we could write about without revealing the details of our\nproprietary research.\n\nplease, feel free to 

In [53]:
ham_json[0].split('\r\n')

[u'message-id: <13247446.1075856567056.javamail.evans#####>',
 u'date: mon, 26 jun 2000 14:22:00 -0700 (pdt)',
 u'from: vkaminski#####',
 u'to: chris#####',
 u'subject: re: eprm articles',
 u'cc: vkaminski#####, vkamins#####',
 u'mime-version: 1.0',
 u'content-type: text/plain; charset=us-ascii',
 u'content-transfer-encoding: 7bit',
 u'bcc: vkaminski#####, vkamins#####',
 u'x-from: vkaminski#####',
 u'x-to: chris#####',
 u'x-cc: vkaminski#####, vkamins#####',
 u'x-bcc: ',
 u'x-origin: kaminski-v',
 u'x-filename: vkamins.nsf',
 u'',
 u"chris,\n\nthanks for the invitation. the evening of july the 18th is fine  with me.\n\nthe list looks fine and it can be easily expanded, if the first set of\narticles is well received. i shall prepare a list of topics that occupy us\nevery day and that we could write about without revealing the details of our\nproprietary research.\n\nplease, feel free to send the message from lacima. i think that it's better\nfor us to sign the articles with our names, 

In [55]:
ham_json[0].replace('\r\n', '\')

u"message-id: <13247446.1075856567056.javamail.evans#####>\ndate: mon, 26 jun 2000 14:22:00 -0700 (pdt)\nfrom: vkaminski#####\nto: chris#####\nsubject: re: eprm articles\ncc: vkaminski#####, vkamins#####\nmime-version: 1.0\ncontent-type: text/plain; charset=us-ascii\ncontent-transfer-encoding: 7bit\nbcc: vkaminski#####, vkamins#####\nx-from: vkaminski#####\nx-to: chris#####\nx-cc: vkaminski#####, vkamins#####\nx-bcc: \nx-origin: kaminski-v\nx-filename: vkamins.nsf\n\nchris,\n\nthanks for the invitation. the evening of july the 18th is fine  with me.\n\nthe list looks fine and it can be easily expanded, if the first set of\narticles is well received. i shall prepare a list of topics that occupy us\nevery day and that we could write about without revealing the details of our\nproprietary research.\n\nplease, feel free to send the message from lacima. i think that it's better\nfor us to sign the articles with our names, giving our respective\naffiliations. in this way, ##### gets the cred

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

